<a href="https://colab.research.google.com/github/cbaldassari/gmm_init/blob/main/workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Workflow

This repository contains the code to reproduce all the results reported in the paper Unsupervised EM Initialization for Mixture Models: A Complex Network Driven Approach for Modeling Financial Time Series.

##Data preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
!pip install community
!pip install python-louvain
!pip install tsia
!pip install networkx
!pip install easydev
!pip install colormap
!pip install missingpy

In [ ]:
import pandas as pd
import numpy as np
from sklearn import mixture
import matplotlib.pyplot as plt
import missingpy
from missingpy import MissForest
import statsmodels.api as sm
from matplotlib import gridspec
from numba import njit, prange
from pyts.image import MarkovTransitionField
import tsia.plot
import tsia.markov
import tsia.network_graph
import community
from community import community_louvain
import networkx as nx
from matplotlib.colors import to_hex
from sklearn.mixture import GaussianMixture
from scipy.stats import kurtosis, skew
import csv
from colormap import rgb2hex
from pickle import FALSE

###Gap Filling, Trend Removal and Log-return transformation

In [ ]:
hubfilled=pd.DataFrame()
hubnames=["gas","oil","spx","pjm"]
stochastic2=pd.DataFrame()
stochastic3=pd.DataFrame()
stochastic4=pd.DataFrame()
logprezzi=pd.DataFrame()
trend=pd.DataFrame()
prezzi=pd.DataFrame()
flagin=True
stocha=[]
chart=pd.DataFrame()
logprezzidepurati=pd.DataFrame()
r=pd.DataFrame()
x=pd.DataFrame()

for hub in hubnames:
  huball=pd.read_excel("market_data/"+hub+".xls",sheet_name="Data 1",parse_dates=["Trade date"])
  huball.rename(columns={'Wtd avg price $/MWh':'Price',  'Trade date':'Date'},inplace=True)
  huballhub=huball.loc[huball['hub_transcode'] == hub]
  a=len(huballhub)
  huballhub.drop(["hub_transcode"],axis=1, inplace=True)
  huballhub=huballhub.groupby('Date').first()
  minrangedate=huballhub.index.min()
  maxrangedate=huballhub.index.max()
  check=pd.Series(pd.date_range(start = (minrangedate), end = maxrangedate, freq='D'))
  check.sort_index(ascending=True)
  #cnt=0
  huballhub = huballhub.reindex(check, fill_value=np.NaN)
  huballhub.index.name = "Date"
  idxs=np.arange(huballhub.shape[0])##
  toimpute=pd.DataFrame()
  toimpute["Date"]=idxs
  toimpute["Price"]=huballhub["Price"].values
  from missingpy import MissForest
  imputer = MissForest(random_state=1234)
  X_imputed = imputer.fit_transform(toimpute.to_numpy(),)
  huballhub["Price"]=X_imputed[:,1]
  df=huballhub
  start = '01-01-2015'
  end = '31-12-2019'
  mask = (huballhub.index >= start) & (huballhub.index <= end)
  huballhub = huballhub.loc[mask]  
  hubfilled[hub]=huballhub["Price"]
  chart[hub]=hubfilled[hub]
  lowess = sm.nonparametric.lowess(np.log(hubfilled[hub]), hubfilled.index, frac=0.1)  
  stocha=np.log(hubfilled[hub])-lowess[:, 1]
  prezzi[hub]=hubfilled[hub]
  logprezzidepurati[hub]=np.log(hubfilled[hub])-lowess[:, 1]
  logprezzi[hub]=np.log(hubfilled[hub])
  if (flagin):
    trend=trend.reindex_like(prezzi)
    flagin=False
  trend[hub]=lowess[:, 1]
  stochastic3[hub]=hubfilled[hub]
  stochastic4[hub]=sm.nonparametric.lowess(hubfilled[hub], hubfilled.index, frac=0.1)[:, 1]
  stochastic2[hub]=stocha-stocha.shift(1)
  x[hub]=stocha
  r[hub]=stochastic2[hub]
stochastic2.dropna(inplace=True) 

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3


###Some helper functions

In [ ]:
# Useful constants definition
COLORMAP = 'jet'

def get_network_graph_f(mtf):
    # Build the graph with networkx:
    graph = nx.from_numpy_matrix(mtf)
    
    # Loops through the edges to get associate each of them with the
    # corresponding Markov transition probability:
    weights = [mtf[u,v] for u,v in graph.edges()]
    for index, e in enumerate(graph.edges()):
        graph[e[0]][e[1]]['weight'] = weights[index]
        
    return graph
    
def compute_network_graph_statistics_f(partitions, graph=None, mtf=None):    
    if (graph is None) and (mtf is not None):
        graph = get_network_graph(mtf)
        
    #partitions = community_louvain.best_partition(graph, random_state=1234)
    nb_partitions = len(set(partitions.values()))
    modularity = community_louvain.modularity(partitions, graph)

    
    diameter = nx.diameter(graph)
    node_size = list(nx.clustering(graph, weight='weight').values())
    avg_clustering_coeff = np.array(node_size).mean()
    density = nx.density(graph)
    avg_path_length = nx.average_shortest_path_length(graph, weight='weight', method='dijkstra')
    
    average_degree = nx.average_degree_connectivity(graph)
    average_degree = np.mean(list(average_degree.values()))
    avg_weighted_degree = nx.average_degree_connectivity(graph, weight='weight')
    avg_weighted_degree = np.mean(list(avg_weighted_degree.values()))
    
    statistics = {
        'Diameter': diameter,
        'Average degree': average_degree,
        'Average weighted degree': avg_weighted_degree,
        'Density': density,
        'Average path length': avg_path_length,
        'Average clustering coefficient': avg_clustering_coeff,
        'Modularity': modularity,
        'Partitions': nb_partitions
    }
    
    return statistics
    
def get_modularity_encoding_f(graph, colormap=COLORMAP, reversed_cmap=False):
  
    if reversed_cmap == True:
        colormap = plt.cm.get_cmap(colormap).reversed()
    else:
        colormap = plt.cm.get_cmap(colormap)
    
    # Get the node partitions and number of partitions found with the Louvain
    # algorithm, as implemented in the `community` package:

    partitions = community_louvain.best_partition(graph, random_state=1234)
    #####################################
    
    nb_partitions = len(set(partitions.values()))
    #print("nb_partitions: ",nb_partitions)

    # Compute node colors and edges colors for the modularity encoding:
    edge_colors = [to_hex(colormap(partitions.get(v)/(nb_partitions - 1))) for u,v in graph.edges()]
    node_colors = [partitions.get(node) for node in graph.nodes()]
    node_size = list(nx.clustering(graph, weight='weight').values())
    node_size = list((node_size - np.min(node_size)) * 2000 + 10)
    
    # Store the encoding to return in a dictionnary:
    #print("node_colors: ",len(set(node_colors)))

    encoding = {
        'node_size': node_size,
        'edge_color': edge_colors,
        'node_color': node_colors
    }
    return encoding, partitions

def stat_f(w,m,v):
  x2=[]
  x3=[]
  x4=[]
  n=len(w)
  for j in range(n):
    x2.append(v[j]+m[j]**2)
    x3.append(pow(m[j],3)+3*m[j]*v[j])
    x4.append(pow(m[j],4)+6*m[j]**2*v[j]+3*v[j]**2)
  X1=np.dot(w,m)
  X2=np.dot(w,x2)
  X3=np.dot(w,x3)
  X4=np.dot(w,x4)
  mu=X1
  sig=np.sqrt(np.subtract(X2, mu**2))
  sk=(X3-3*X2*X1+2*pow(X1,3))/pow(sig,3)
  kur=(X4-4*X3*X1+6*X2*X1**2-3*pow(X1,4))/pow(sig,4)
  return [mu, sig, sk, kur]

def get_network_graph_map_f(timeseries, encoding, colormap=COLORMAP, reversed_cmap=False):
   
    # Get encoding definitions:
    node_colors = encoding['node_color']

    #print(node_colors)

    image_size = len(node_colors)
    #print("node_colors",node_colors)
    #print("np.max(node_colors)",np.max(node_colors))
    partition_color = node_colors / np.max(node_colors)

    # Define the color map:
    if reversed_cmap == True:
        colormap = plt.cm.get_cmap(colormap).reversed()
    else:
        colormap = plt.cm.get_cmap(colormap)

    # Plot each subset of the signal with the color associated to the network
    # graph partition it belongs to:
    network_graph_map = []
    sequences_width = timeseries.shape[0] / image_size

    #df=pd.DataFrame([{"color": p ,"value": k}])

    for i in range(image_size):
        c = colormap(partition_color[i])

        start = int(i * sequences_width)
        end = int((i+1) * sequences_width)#-1
        data = timeseries.iloc[start:end, :]

        current_map = dict()

        current_map.update({
            'color': c,
            'slice': data
        })

        #print(len(current_map["slice"]))

        network_graph_map.append(current_map)
        
    return network_graph_map, node_colors


def inversemapAna(ng_map2,colors2):
  df=pd.DataFrame(columns=["color","value"])
  dout=pd.DataFrame(columns=["color","value"])
  for i in range(len(ng_map2)):
      d=ng_map2[i]
      p=colors2[i]
      slic=d["slice"].values.reshape(-1)

      for k in slic:
        df=df.append([{"color": p ,"value": k}], ignore_index=True)
  
  df["diff"]=df["value"]-df["value"].shift(1)
  df.drop(df.index[[0]], inplace=True)
  df.drop(['value'], axis = 1, inplace=True)
  df.rename(columns = {'diff':'value'}, inplace = True)
  return df

##Distribution estimation

### Markov Transition Fields, Network association, Network stats, Gaussian mixture model and Optimization Grid 

In [ ]:
hubnames=["pjm","gas","oil","spx"]
for hub in hubnames:
    r = pd.read_pickle("returns.pk")
    rr=r[hubnames].dropna()
    flagFirst=True
    strategy = 'quantile'
    note=pd.DataFrame(columns=["hub","bins","imsize","bic","aic"])
    flag=True
    gridlist = []
    for b in range(2,55,2):
      for ts in range(5,405,5):
        gridlist.append((b,ts))
    gridlist = tuple(gridlist)

    ccc=0
    tag_data = tag_data = pd.read_pickle("logdetrend.pk")
    tag_df=tag_data[hub]
    for g in gridlist:
      bins=g[0]
      imsize=g[1]

    X = tag_df.values.reshape(1, -1)

    mtf = MarkovTransitionField(image_size=imsize, n_bins=bins, strategy=strategy,overlapping=False)
    tag_mtf = mtf.fit_transform(X)
    
    graph= get_network_graph_f(tag_mtf[0])
    encoding2, partitions = get_modularity_encoding_f(graph)
    
    ccc+=1
    if (len(set(partitions.values()))<=10):
      tag_df=pd.DataFrame(tag_df)
    
      ng_map2, colors2 = get_network_graph_map_f(tag_df, encoding2)
      statistics=compute_network_graph_statistics_f(partitions,graph)

      nb_partitions=statistics["Partitions"]
      modularity=statistics["Modularity"]

      lin_map=inversemapAna(ng_map2,colors2)      
      tag_df=rr

      colors=lin_map.groupby(['color']).size()
      colors=colors.index

      means=[]
      precisions=[]
      nk=[]
        
      for c in colors:
        a=lin_map.value[lin_map["color"]==c].values
        means.append(np.mean(a))
        precisions.append(1/pow(np.std(a),2))
        nk.append(len(a)/len(lin_map))

      precisions=np.array(precisions).reshape(-1,1,1)
      means=np.array(means).reshape(-1,1)

      grid=pd.DataFrame(columns=["idxs","hub","aic","bic","comp","weights","means","covariances"])
      grid.set_index("idxs")     
      
      itemorig={"1":tag_df[hub].mean(),
            "2":tag_df[hub].std(),
            "3":skew(tag_df[hub]),
            "4":kurtosis(tag_df[hub])+3
      }

      XY = tag_df[hub].values.reshape(-1, 1)

      try:
          gmm = GaussianMixture(n_components=len(nk), weights_init=nk, means_init=means, precisions_init=precisions, covariance_type='full').fit(XY)
      except ValueError:      
        continue
        
      nosim=foo(gmm.weights_.reshape(-1),gmm.means_.reshape(-1),gmm.covariances_.reshape(-1))

      grid.at[0,'comp']=nb_partitions
      grid.at[0,'hub']=hub
      grid.at[0,'comp']=gmm.n_components

      grid.at[0,'bins']=bins
      grid.at[0,'imsize']=imsize
      grid.at[0,'netstat']=str(statistics)

      grid.at[0,'bic']=gmm.bic(XY) 
      grid.at[0,'aic']=gmm.aic(XY)
      grid.at[0,'weights']=gmm.weights_.reshape(-1)
      grid.at[0,'means']=gmm.means_.reshape(-1)
      grid.at[0,'covariances']=gmm.covariances_.reshape(-1)

      grid.at[0,'orig_M2']=itemorig["2"]
      grid.at[0,'orig_M3']=itemorig["3"]
      grid.at[0,'orig_M4']=itemorig["4"]

      grid.at[0,'GMM_M2']=nosim[1]
      grid.at[0,'GMM_M3']=nosim[2]
      grid.at[0,'GMM_M4']=nosim[3]

      grid.at[0,'absdiff_M2']=abs(itemorig["2"]-nosim[1])
      grid.at[0,'absdiff_M3']=abs(itemorig["3"]-nosim[2])
      grid.at[0,'absdiff_M4']=abs(itemorig["4"]-nosim[3])

      grid.at[0,'rel%diff_M2']=100*abs((itemorig["2"]-nosim[1])/(itemorig["2"]))
      grid.at[0,'rel%diff_M3']=100*abs((itemorig["3"]-nosim[2])/(itemorig["3"]))
      grid.at[0,'rel%diff_M4']=100*abs((itemorig["4"]-nosim[3])/(itemorig["4"]))
      
      if (flag):
        grid.to_csv("/content/drive/MyDrive/Mari/plotpaper2/x_r_"+strategy+"_MTFgrid_"+hub.upper()+".csv", header=True, mode='a')
        flag=False
      else:
        grid.to_csv("/content/drive/MyDrive/Mari/plotpaper2/x_r_"+strategy+"_MTFgrid_"+hub.upper()+".csv", header=False, mode='a')

      print(bins,"#",imsize)